In [21]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as F
from pyspark import SparkContext, SparkConf

In [22]:
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark_sql.SparkSession.builder.getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\DELL\AppData\Local\Temp\ipykernel_11144\2229581302.py:2 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dense_rank,desc
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("FIFA").getOrCreate()


In [ ]:
df = spark.read.csv('california_cities.csv',header=True)
df=df.drop(F.col('_c0'))


In [23]:
df.show()

+--------------+------------------+-------------------+-----------+------------+----------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|          city|              latd|              longd|elevation_m|elevation_ft|population_total|  area_total_sq_mi|   area_land_sq_mi|    area_water_sq_mi|    area_total_km2|     area_land_km2|      area_water_km2|
+--------------+------------------+-------------------+-----------+------------+----------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|      Adelanto| 34.57611111111112|-117.43277777777779|      875.0|      2871.0|           31765|            56.027| 56.00899999999999|0.018000000000000002|           145.107|           145.062|               0.046|
|   AgouraHills| 34.15333333333333|-118.76166666666667|      281.0|       922.0|           20330|             7.822| 7.792999999999999|0

1. Find out the missing values in each column and store them in the variable col_missing_values

In [24]:
col_missing_values={}
from pyspark.sql.functions import col,isnan,when,count
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+----+----+-----+-----------+------------+----------------+----------------+---------------+----------------+--------------+-------------+--------------+
|city|latd|longd|elevation_m|elevation_ft|population_total|area_total_sq_mi|area_land_sq_mi|area_water_sq_mi|area_total_km2|area_land_km2|area_water_km2|
+----+----+-----+-----------+------------+----------------+----------------+---------------+----------------+--------------+-------------+--------------+
|   0|   0|    0|         48|          12|               0|               2|              0|               1|             5|            4|             4|
+----+----+-----+-----------+------------+----------------+----------------+---------------+----------------+--------------+-------------+--------------+



In [25]:
df.describe().show()

+-------+-----------+------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+
|summary|       city|              latd|              longd|      elevation_m|      elevation_ft|  population_total|  area_total_sq_mi|   area_land_sq_mi|  area_water_sq_mi|   area_total_km2|    area_land_km2|    area_water_km2|
+-------+-----------+------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+
|  count|        482|               482|                482|              434|               470|               482|               480|               482|               481|              477|              478|               478|
|   mean|       NULL|36.095088081604395|-119.83771392577232|180.4167050691244| 567.1

Find the total missing values in the whole dataset and store the number in df_missing_values

In [26]:
first_row = df2.head()  # Get the first row as a Row object
sum_first_row = sum(first_row)  # Sum the values of the first row
sum_first_row

76

In [27]:
i = 0
sum_value = 0  # Changed variable name from 'sum' to 'sum_value' to avoid conflict with built-in 'sum' function
for c in range(len(df2.columns)):
    sum_value += df2.collect()[0][i]
    i += 1

print(sum_value)
    

76



Drop the column area_water_percent as it has the most of its values missing

In [28]:
df= df.drop('area_water_percent')

In [29]:
df.show()

+--------------+------------------+-------------------+-----------+------------+----------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|          city|              latd|              longd|elevation_m|elevation_ft|population_total|  area_total_sq_mi|   area_land_sq_mi|    area_water_sq_mi|    area_total_km2|     area_land_km2|      area_water_km2|
+--------------+------------------+-------------------+-----------+------------+----------------+------------------+------------------+--------------------+------------------+------------------+--------------------+
|      Adelanto| 34.57611111111112|-117.43277777777779|      875.0|      2871.0|           31765|            56.027| 56.00899999999999|0.018000000000000002|           145.107|           145.062|               0.046|
|   AgouraHills| 34.15333333333333|-118.76166666666667|      281.0|       922.0|           20330|             7.822| 7.792999999999999|0

Drop the rows having missing values and store the resulting DataFrame in the variable df_narows_dropped

In [30]:
df_narows_dropped=df.dropna()

Drop Rows with More Than 5 Missing Values

In [31]:
df_rows_dropped=df.na.drop(how='any', thresh=5, subset=None)

Eliminate the columns in the DataFrame that contain missing values. Store the resulting DataFrame in the variable df_nacols_dropped.

In [32]:
df3 = df.select([

  count(when(F.col(co).isNull(),co)).alias(co)  
for co in df.columns
])

df3.collect()[0]
bag=[]
idx=0
df_nacols_dropped=df
for idx in range(len(df.columns)):
    if df3.collect()[0][idx]!=0:
        bag.append(df.columns[idx])
for el in bag:
   df_nacols_dropped=df_nacols_dropped.drop(el) 

In [33]:
df_nacols_dropped= df_nacols_dropped.select([

  count(when(F.col(co).isNull(),co)).alias(co)  
for co in df_nacols_dropped.columns
])
df_nacols_dropped.show()

+----+----+-----+----------------+---------------+
|city|latd|longd|population_total|area_land_sq_mi|
+----+----+-----+----------------+---------------+
|   0|   0|    0|               0|              0|
+----+----+-----+----------------+---------------+



Fill the 50 missing values in elevation_m with -999. Store your result in the variable filled_elevation_m

In [34]:
print(df.select('elevation_ft').describe().show())
df4=df.na.fill(value=6,subset=['elevation_ft'])

+-------+------------------+
|summary|      elevation_ft|
+-------+------------------+
|  count|               470|
|   mean| 567.1687234042553|
| stddev|1001.2657219060482|
|    min|            -180.0|
|    max|            7880.0|
+-------+------------------+

None


In [35]:
df4.select('elevation_ft').describe().show()

+-------+-----------------+
|summary|     elevation_ft|
+-------+-----------------+
|  count|              482|
|   mean|553.1977178423236|
| stddev|992.5636617124128|
|    min|           -180.0|
|    max|           7880.0|
+-------+-----------------+



In [36]:

# Check the schema of the DataFrame
df.printSchema()

# If the column is not of the correct numeric type, cast it
df = df.withColumn("elevation_ft", col("elevation_ft").cast("double"))

# Verify schema after casting
df.printSchema()

# Fill null values in the 'elevation_ft' column with a numeric value, e.g., 6
df3 = df.na.fill(value=699933, subset=['elevation_ft'])

# Print description after filling nulls
print("Description after filling nulls:")
df3.select('elevation_ft').describe().show()

root
 |-- city: string (nullable = true)
 |-- latd: string (nullable = true)
 |-- longd: string (nullable = true)
 |-- elevation_m: string (nullable = true)
 |-- elevation_ft: double (nullable = true)
 |-- population_total: string (nullable = true)
 |-- area_total_sq_mi: string (nullable = true)
 |-- area_land_sq_mi: string (nullable = true)
 |-- area_water_sq_mi: string (nullable = true)
 |-- area_total_km2: string (nullable = true)
 |-- area_land_km2: string (nullable = true)
 |-- area_water_km2: string (nullable = true)

root
 |-- city: string (nullable = true)
 |-- latd: string (nullable = true)
 |-- longd: string (nullable = true)
 |-- elevation_m: string (nullable = true)
 |-- elevation_ft: double (nullable = true)
 |-- population_total: string (nullable = true)
 |-- area_total_sq_mi: string (nullable = true)
 |-- area_land_sq_mi: string (nullable = true)
 |-- area_water_sq_mi: string (nullable = true)
 |-- area_total_km2: string (nullable = true)
 |-- area_land_km2: string (null

In [40]:
from pyspark.sql.functions import mean
#define function to fill null values with column mean
def fillna_mean(df, include=set()): 
    means = df.agg(*(
        mean(x).alias(x) for x in df.columns if x in include
    ))
# getting the first row and then transforming it into dictionary and then mapping
    return df.fillna(means.first().asDict())

#fill null values with mean in specific columns
df = fillna_mean(df, ['area_land_km2', 'latd'])


In [41]:
include=df.columns
means = df.agg(*(
        F.mean(x).alias(x) for x in df.columns if x in include 
    ))


In [42]:
means.show()

+----+------------------+-------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+
|city|              latd|              longd|      elevation_m|     elevation_ft|  population_total|  area_total_sq_mi|   area_land_sq_mi|  area_water_sq_mi|   area_total_km2|    area_land_km2|   area_water_km2|
+----+------------------+-------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+
|NULL|36.095088081604395|-119.83771392577232|180.4167050691244|567.1687234042553|64894.933609958505|18.442183333333325|16.840981327800833|1.5896299376299374|46.75328301886789|43.54702092050211|3.125178242677823|
+----+------------------+-------------------+-----------------+-----------------+------------------+------------------+------------------+--------------